Most AI of autonmous vehicles (AVs) comes not standalone but rather with a large number of rules ans heuristics that restrict what actions the AI takes or can take. This primarily relates to managing risks, such as those resulting from collisions or crashes.

There are number of standard risk measures that are typically used, such as (trailing) stop loss orders or take profit orders. How such risk measures are handles is a matter of human judgement. AI provides improved predictions, but human judgement still plays a role in setting decision rules and action boundaries.

First, it backtests in both vectorized and event-based fashio algorithm trading strategies that result from a trained deep Q-learning agent called trading bots.

Second, it assesses risks related to the financial instrument on which the strategies are implemented on which the strategies are implemented.

Third, it backtests typical risk measures using the event-based approach. The major benefit of event-based backtesting when compared to vectorized backtesting is higher degree of flexibility in modeling and analyzing decision rule and risk management measures.

# Trading Bot

This section presents a trading bot based on the financial Q-learning agent, `FQLAgent`.

In [1]:
import os
import numpy as np
import pandas as pd
from pylab import plt, mpl

We use `finance` class and `tradingbot` class